****
## JSON mini project

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated before (examples removed for this notebook),
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [112]:
# Import data science packages
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize


In [52]:
# Import the project dataset from a JSON file
jsout = json.load((open('data/world_bank_projects.json')))

# Get the messy JSON
raw_df = pd.read_json('data/world_bank_projects.json')

## 1. Find the 10 countries with most projects

In [60]:
# Merge normalized DataFrame with the raw one 
top10_countries = raw_df
top10_countries = top10_countries[['id','countryname']]

# Get unique project IDs, count by country, return top 10
top10_countries = top10_countries.drop_duplicates()
top10_countries = top10_countries.countryname.value_counts()
top10_countries = top10_countries.head(10)
top10_countries

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Nepal                              12
Kingdom of Morocco                 12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

 ## 2. Find the top 10 major project themes (using column 'mjtheme_namecode')

In [62]:
# Normalize the raw JSON file on the 'mjtheme_namecode' column,
#     include 'id' for merging
normal_df = pd.DataFrame(json_normalize(jsout,'mjtheme_namecode','id'))

top10_themes = normal_df
top10_themes = top10_themes['name'].value_counts()
top10_themes.head(10)
top10_themes

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Rule of law                                      12
Name: name, dtype: int64

Note that there is a blank project theme name in the top 10! We'll find the correct numbers in the 3rd part of this project.

## 3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [65]:
# Replace all empty strings with NaN to make replacement easier
fixed_df = normal_df.replace('',np.nan)

# Perform a ffill and a bfill to ensure correct value is filled from before or after the current row
fixed_df = fixed_df.groupby('name').ffill().bfill()

# Provide new names to reflect the normalized columns
fixed_df.columns = ['mjtheme_code','mjtheme_name','id']

In [217]:
# Merge fixed_df with the original raw_df
clean_df = raw_df.merge(fixed_df, on='id')

# Remove old, unnormalized column
clean_df = clean_df.drop('mjtheme_namecode', axis=1)

Let's find the correct numbers for the top 10 major project themes, having gotten the blank project theme entries accounted for!

In [67]:
# Sort the new DataFrame by mjproject
clean_top10_themes = clean_df['mjtheme_name']

#And here are the actual numbers! 
clean_top10_themes.value_counts().head(10)

Environment and natural resources management    250
Human development                               218
Rural development                               215
Public sector governance                        198
Social protection and risk management           177
Financial and private sector development        142
Social dev/gender/inclusion                     126
Trade and integration                            78
Urban development                                49
Economic management                              34
Name: mjtheme_name, dtype: int64

## Just out of curiosity, let's find out how much these counts changed between the wrong version and the right version!

In [216]:
# We're not going to compare the empty-named categories
wrong_df = top10_themes.sort_index().drop('')
wrong_df

Economic management                              33
Environment and natural resources management    223
Financial and private sector development        130
Human development                               197
Public sector governance                        184
Rule of law                                      12
Rural development                               202
Social dev/gender/inclusion                     119
Social protection and risk management           158
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

In [192]:
right_df = clean_top10_themes.value_counts().sort_index()
right_df

Economic management                              34
Environment and natural resources management    250
Financial and private sector development        142
Human development                               218
Public sector governance                        198
Rule of law                                      12
Rural development                               215
Social dev/gender/inclusion                     126
Social protection and risk management           177
Trade and integration                            78
Urban development                                49
Name: mjtheme_name, dtype: int64

In [215]:
diff = pd.DataFrame(right_df - wrong_df)
diff.columns = ['absolute error']
diff['percent error'] = (diff['absolute error'] / right_df * 100).map('{:,.1f}%'.format)
diff

,absolute error,percent error
Economic management,1,2.9%
Environment and natural resources management,27,10.8%
Financial and private sector development,12,8.5%
Human development,21,9.6%
Public sector governance,14,7.1%
Rule of law,0,0.0%
Rural development,13,6.0%
Social dev/gender/inclusion,7,5.6%
Social protection and risk management,19,10.7%
Trade and integration,6,7.7%


The percent error, calculated by dividing the absolute error by the correct value, shows just how important correctly filling missing data can be. Especially with the Environmental and natural resources management and Social protection and risk management themes, we would have been off by over 10 percent had we not taken care of those missing values!